# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import urllib
import re

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [2]:
# your code here
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=0&sort=popularity'
headers = {
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
           
          }

r = requests.get(url, headers=headers)
results = r.json()
results
x = json_normalize(results)
data = json_normalize(x.articles[0])
data


,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,...,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,tracking_information.metrigo_impression_urls,tracking_information.impression_beacon,tracking_information.source,outfits,amount,price.base_price
0,PI922S03M-M11,Sweat à capuche - dark green,"[XXS, XS, S, M, L, XL]",pier-one-sweatshirt-dark-green-pi922s03m-m11,[{'path': 'PI/92/2S/03/MM/11/PI922S03M-M11@17....,Pier One,False,"[{'sku': 'PI922S03M-M11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,[https://ccp-et.metrigo.zalan.do/event/sbv?z=c...,https://ccp-et.metrigo.zalan.do/event/sbv?z=cd...,ccp,NaN,NaN,NaN
1,PI922S049-Q11,Sweatshirt - black,"[XS, S, M, L, XL, XXL]",pier-one-sweatshirt-black-pi922s049-q11,[{'path': 'PI/92/2S/04/9Q/11/PI922S049-Q11@9.j...,Pier One,False,"[{'sku': 'PI922S049-Q11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,...,False,False,False,False,[https://ccp-et.metrigo.zalan.do/event/sbv?z=c...,https://ccp-et.metrigo.zalan.do/event/sbv?z=cd...,ccp,NaN,NaN,NaN
2,PI922S03J-E11,Sweat à capuche - yellow,"[XS, S, M, L, XL]",pier-one-sweatshirt-yellow-pi922s03j-e11,[{'path': 'PI/92/2S/03/JE/11/PI922S03J-E11@8.j...,Pier One,False,"[{'sku': 'PI922S03J-E11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,[https://ccp-et.metrigo.zalan.do/event/sbv?z=c...,https://ccp-et.metrigo.zalan.do/event/sbv?z=cd...,ccp,NaN,NaN,NaN
3,L0642D008-Q11,HERREN - T-shirt basique - black,"[L, XL, XXL, 3XL]",lacoste-sport-t-shirt-de-sport-black-l0642d008...,[{'path': 'L0/64/2D/00/8Q/11/L0642D008-Q11@16....,Lacoste Sport,False,"[{'sku': 'L0642D008-Q11', 'url_key': 'lacoste-...","[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,TOB22O01B-Q11,ORIGINAL SLIM FIT - T-shirt à manches longues ...,"[XS, S, L, XL, XXL]",tommy-jeans-original-t-shirt-a-manches-longues...,[{'path': 'TO/B2/2O/01/BQ/11/TOB22O01B-Q11@8.j...,Tommy Jeans,False,"[{'sku': 'TOB22O01B-Q11', 'url_key': 'tommy-je...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,YO122E00V-N11,Pantalon cargo - olive,"[28, 33, 34, 36]",yourturn-pantalon-cargo-olive-yo122e00v-n11,[{'path': 'YO/12/2E/00/VN/11/YO122E00V-N11@10....,YOURTURN,False,"[{'sku': 'YO122E00V-N11', 'url_key': 'yourturn...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
80,NI122F01J-Q11,CLUB - Pantalon de survêtement - schwarz/weiß,"[XS, S, M, L, XL, XXL]",nike-sportswear-pantalon-de-survetement-schwar...,[{'path': 'NI/12/2F/01/JQ/11/NI122F01J-Q11@10....,Nike Sportswear,False,"[{'sku': 'NI122F01J-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
81,PI952H02T-Q11,Sac week-end - black,[One Size],pier-one-sac-week-end-black-pi952h02t-q11,[{'path': 'PI/95/2H/02/TQ/11/PI952H02T-Q11@10....,Pier One,False,"[{'sku': 'PI952H02T-Q11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-70%', 'tra...",accessoires,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
82,YO122Q03F-Q11,Pullover - anthracite,"[S, M, L, XL, XXL]",yourturn-pullover-anthracite-yo122q03f-q11,[{'path': 'YO/12/2Q/03/FQ/11/YO122Q03F-Q11@8.j...,YOURTURN,False,"[{'sku': 'YO122Q03F-Q11', 'url_key': 'yourturn...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN


## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [3]:
# your code here
total_page = results['pagination']['page_count']
# Your code
lst = []
for i in range(total_page):
    y = i*84
    url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset='+str(y)+'&sort=popularity'

    r = requests.get(url, headers=headers)
    results = r.json()
    x = json_normalize(results)
    y = json_normalize(x.articles[0])
    lst.append(y)
result = pd.concat(lst,sort=False, ignore_index=True)

In [17]:
result

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,...,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,tracking_information.metrigo_impression_urls,tracking_information.impression_beacon,tracking_information.source,outfits,amount,price.base_price
0,PI922S03M-M11,Sweat à capuche - dark green,"[XXS, XS, S, M, L, XL]",pier-one-sweatshirt-dark-green-pi922s03m-m11,[{'path': 'PI/92/2S/03/MM/11/PI922S03M-M11@17....,Pier One,False,"[{'sku': 'PI922S03M-M11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,[https://ccp-et.metrigo.zalan.do/event/sbv?z=f...,https://ccp-et.metrigo.zalan.do/event/sbv?z=fa...,ccp,NaN,NaN,NaN
1,PI922S049-Q11,Sweatshirt - black,"[XS, S, M, L, XL, XXL]",pier-one-sweatshirt-black-pi922s049-q11,[{'path': 'PI/92/2S/04/9Q/11/PI922S049-Q11@9.j...,Pier One,False,"[{'sku': 'PI922S049-Q11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,...,False,False,False,False,[https://ccp-et.metrigo.zalan.do/event/sbv?z=f...,https://ccp-et.metrigo.zalan.do/event/sbv?z=fa...,ccp,NaN,NaN,NaN
2,PI922Q058-O11,Gilet - mottled brown,"[S, M, L, XL, XXL]",pier-one-gilet-mottled-brown-pi922q058-o11,[{'path': 'PI/92/2Q/05/8O/11/PI922Q058-O11@10....,Pier One,False,"[{'sku': 'PI922Q058-O11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,...,False,False,False,False,[https://ccp-et.metrigo.zalan.do/event/sbv?z=f...,https://ccp-et.metrigo.zalan.do/event/sbv?z=fa...,ccp,"[{'id': 'BYdXjjC8TDS', 'url_key': '/outfits/BY...",NaN,NaN
3,AD115B01K-A12,STAN SMITH STREETWEAR-STYLE SHOES - Baskets ba...,"[36, 38, 40, 42, 44, 46, 48, 50, 36 2/3, 37 1/...",adidas-originals-stan-smith-baskets-basses-ad1...,[{'path': 'AD/11/5B/01/KA/12/AD115B01K-A12@12....,adidas Originals,False,"[{'sku': 'AD115B01K-A12', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': '-10%', 'tra...",shoe,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,C1422E00D-N14,MADISON JOGGER TRABUCO - Pantalon classique - ...,"[S, M, L, XL, XXL]",carhartt-wip-madison-trabuco-pantalon-classiqu...,[{'path': 'C1/42/2E/00/DN/14/C1422E00D-N14@4.j...,Carhartt WIP,False,"[{'sku': 'C1422E00D-N14', 'url_key': 'carhartt...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51775,PX842L000-Q11,OBEX SPIN - Casque - uranium black,[51-54],poc-obex-spin-casque-uranium-black-px842l000-q11,[{'path': 'PX/84/2L/00/0Q/11/PX842L000-Q11@15....,POC,False,"[{'sku': 'PX842L000-Q11', 'url_key': 'poc-obex...","[{'key': 'discountRate', 'value': '-15%', 'tra...",equipment,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
51776,ES182C003-G11,MIT KARO - Caleçon - red,[M],esprit-mit-karo-calecon-red-es182c003-g11,[{'path': 'ES/18/2C/00/3G/11/ES182C003-G11@13....,Esprit,False,"[{'sku': 'ES182C003-G11', 'url_key': 'esprit-m...","[{'key': 'discountRate', 'value': '-20%', 'tra...",underwear,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
51777,PX844E00E-Q11,AURIC CUT - Casque - matt black,[51-54],poc-auric-cut-casque-px844e00e-q11,[{'path': 'PX/84/4E/00/EQ/11/PX844E00E-Q11@13....,POC,False,"[{'sku': 'PX844E00E-Q11', 'url_key': 'poc-auri...","[{'key': 'discountRate', 'value': '-20%', 'tra...",equipment,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
51778,N1244E0RM-G11,PARIS ST GERMAIN MERC - Protège-tibias - infra...,"[L, XL]",nike-performance-paris-st-germain-merc-protege...,[{'path': 'N1/24/4E/0R/MG/11/N1244E0RM-G11@10....,Nike Performance,False,"[{'sku': 'N1244E0RM-G11', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': '-20%', 'tra...",equipment,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [5]:
result['brand_name'].value_counts()

Pier One                1933
Jack & Jones            1437
BOSS                    1103
Nike Performance        1098
Seidensticker            823
                        ... 
Wildwuchs Bartpflege       1
Pregis                     1
pure cashmere              1
Mackintosh                 1
AND1                       1
Name: brand_name, Length: 810, dtype: int64

In [54]:
name_maximum_discount = ""

max_discount = 0

for i in result.index:
    
    x = re.findall('\d? ?\d+,\d+',result['price.original'][i])
    y = re.findall('\d? ?\d+,\d+',result['price.promotional'][i])

    x = re.sub(',','.',*x)
    y = re.sub(',','.',*y)
    x = str(x)
    y = str(y)
    x = re.sub(' ','',x)
    y = re.sub(' ','',y)   
    
    if abs(float(x) - float(y)) > max_discount:
        print(i)
        print(x)
        max_discount = abs(float(x) - float(y))
        name_maximum_discount = result['name'][i] 
        
print(name_maximum_discount)
print(max_discount)

0
29.95
1
27.95
2
44.95
4
99.95
12
249.95
18
229.95
283
489.95
1229
599.95
4835
394.95
7473
399.95
43426
184.95
45239
169.95
45300
074.95
CLASSIC CAFE RACER - Veste en cuir - black
785.0


In [43]:
print(result['price.original'][45300])

print(result['price.promotional'][45300])

1 074,95 €
859,95 €


In [11]:
# your code here
for i in 

BOSS
Las marcas sin descuentos son ['Fusalp', 'Baldessarini']


In [51]:
a = '1 000'
a = re.sub(" ",'',a)
a

'1000'